In [27]:
import numpy as np
import itertools
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import sys

sys.path.append("../")

import abus_classification

In [2]:
dataset = abus_classification.datasets.TDSCTumors(path="../data/tdsc")    

In [21]:
signature_dataset = []

for i in tqdm(range(len(dataset))):
    _,x,y = dataset[i]
    sig = abus_classification.utils.features.boundary_signature_3d(x, resolution=(5,5))
    signature_dataset.append((sig.flatten(), y))

  0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
X, Y = zip(*signature_dataset)
X = list(X)
Y = list(Y)

In [23]:
def classify_with_svm(x, y):
    
    acc = 0
    train_acc = 0
    cfm = [[0,0],
           [0,0]]

    for i in range(100):
        x_test, y_test = x[i], y[i]
        X_train, Y_train = x[:i] + x[i+1:], y[:i] + y[i+1:]
        clf = SVC()
        clf.fit(X_train, Y_train)
        P = clf.predict(X_train)
        train_acc += accuracy_score(Y_train, P)
        res = clf.predict([x_test])[0]
        if res == y_test:
            acc += 1
            
        cfm[y_test][res] += 1
            
    print(f"Train accuracy: {train_acc/100}")
    print(f"Accuracy: {acc/100}")
    print(f"{cfm[0]}\n{cfm[1]}")

In [28]:
classify_with_svm(X,Y)

Train accuracy: 0.7259595959595961
Accuracy: 0.68
[44, 14]
[18, 24]


In [29]:
def normalize(x):
    for i in range(100):
        x[i] = abus_classification.utils.math.normalize(x[i])
    return x
X = normalize(X)


In [ ]:
print(max(X[0]))

In [30]:
classify_with_svm(X,Y)


Train accuracy: 0.7259595959595961
Accuracy: 0.68
[44, 14]
[18, 24]


In [ ]:
signature_dataset = []

for i in tqdm(range(len(dataset))):
    _,x,y = dataset[i]
    sig = abus_classification.utils.features.boundary_signature_3d(x, resolution=(5,5))
    signature_dataset.append((sig.flatten(), y))
    
